In [13]:
#import libraries
import urllib , json , requests
import pandas as pd
import sqlite3
from sqlite3 import Error
import logging
import os
import pathlib

In [8]:
#all the companies for which we have to do a databse entry
company_name=['MSFT','ABB','AAL','AAPL','DELL']

In [9]:
#get the list of uRL's from where we have to fetch data 
def getURL(arr):
    urlList=[]
    for company in arr:
        url=f"https://www.alphavantage.co/query?function=TIME_SERIES_DAILY_ADJUSTED\
        &apikey=NQCFKOVGZASY3EZ9&symbol={company}"
        url = url.replace(" ","") #in case there is any space in URL 
        urlList.append(url)
    return urlList

In [10]:
#imports json from the Restapi 
def ImportJson(url):
    response =urllib.request.urlopen(url)
    data=json.loads(response.read()) #converts to python dict 
    return data

In [ ]:
def createConnection(db_file):
    

In [11]:
#create table 
def CreateTable(conn,query):
    try:
        cursor=conn.cursor()
        cursor.execute(query)
    except Error as e:
        print(e)

In [14]:
print(pathlib.Path.cwd())

C:\Users\saksham.gupta\assignment\stock_data_skasham
